# 개요

# Prepare

# 1. 초기 세팅

## Library & data load

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [2]:
# 처리용 df와 오리지널 버전 남겨놓음
df = pd.read_csv("/Users/luci031/Downloads/auction/data_origin/auction_202202.csv",encoding='euc_kr')
org_df = pd.read_csv("/Users/luci031/Downloads/auction/data_origin/auction_202202.csv",encoding='euc_kr')

# Pre-Processing

## Basic info

In [3]:
df.head()

,품목명,단위,등급,가격,산지,친환경구분,입력일
0,[대파]대파(일반),1kg,특(1등),900,전남,일반,20220204
1,[만감]레드향,5kg,특(1등),12000,제주,일반,20220204
2,[무]마대무,20kg,특(1등),6500,제주,일반,20220204
3,[무]마대무,20kg,상(2등),5200,제주,일반,20220204
4,[무]마대무,20kg,상(2등),4800,제주,일반,20220204


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 691646 entries, 0 to 691645
Data columns (total 7 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   품목명     691646 non-null  object
 1   단위      691646 non-null  object
 2   등급      691646 non-null  object
 3   가격      691646 non-null  int64 
 4   산지      690768 non-null  object
 5   친환경구분   691646 non-null  object
 6   입력일     691646 non-null  int64 
dtypes: int64(2), object(5)
memory usage: 36.9+ MB


In [5]:
# 산지 제외하고는 결측치 없는 것으로 확인
df.isnull().sum()

품목명        0
단위         0
등급         0
가격         0
산지       878
친환경구분      0
입력일        0
dtype: int64

- 결측치 처리

In [6]:
###
# 양이 많지 않으므로 결측치 제거
df = df.dropna(axis=0)

- 영문명으로 전환

In [7]:
df = df.rename(columns = {'품목명':'prd',
                          '단위':'scale','등급':'class',
                          '가격':'price',
                          '산지':'origin',
                          '친환경구분':'eco',
                          '입력일':'reg_date'})

## column 별 정보

### class

In [8]:
# 이름 모두 영어로 변경
# 9등과 없음 모두 9로 변경
df['class'].value_counts().index

Index(['특(1등)', '상(2등)', '보통(3등)', '9등(등외)', '없음', '5등', '4등', '6등', '7등',
       '8등'],
      dtype='object')

In [9]:
df111 = pd.DataFrame([['특(1등)', 1], ['상(2등)',2], ['보통(3등)',3], ['9등(등외)',9], ['없음',9], ['5등',5], ['4등',4], ['6등',6], ['7등',7],['8등',8]],columns=['class','new_class'])

In [10]:
df2 = df.merge(df111)
df2 = df2.drop(columns='class')

In [11]:
df2

,prd,scale,price,origin,eco,reg_date,new_class
0,[대파]대파(일반),1kg,900,전남,일반,20220204,1
1,[만감]레드향,5kg,12000,제주,일반,20220204,1
2,[무]마대무,20kg,6500,제주,일반,20220204,1
3,[가지]가지,5kg,14000,경남 밀양시,일반,20220204,1
4,[감]부유단감,10kg,30000,광주 남구,일반,20220204,1
...,...,...,...,...,...,...,...
690763,[호박]늙은호박,8kg,3000,전남 해남군,일반,20220224,8
690764,[양파]양파(일반),15kg,3500,전남 무안군,일반,20220225,8
690765,[양파]양파(일반),15kg,3500,전남 무안군,일반,20220225,8
690766,[토마토]대저토마토,2.5kg,6000,경남 김해시,일반,20220226,8


### eco

In [12]:
# 영문명으로 전환
# 전환기 -> 유기농 재배로 전환중
df2.loc[df2["eco"]=="일반","eco"] = "normal"
df2.loc[df2["eco"]=="우수농산물","eco"] = "good_prd"
df2.loc[df2["eco"]=="무농약","eco"] = "ecofriendly"
df2.loc[df2["eco"]=="유기농","eco"] = "organic"
df2.loc[df2["eco"]=="품질인증","eco"] = "certified"
df2.loc[df2["eco"]=="전환기","eco"] = "ing"

In [13]:
df2

,prd,scale,price,origin,eco,reg_date,new_class
0,[대파]대파(일반),1kg,900,전남,normal,20220204,1
1,[만감]레드향,5kg,12000,제주,normal,20220204,1
2,[무]마대무,20kg,6500,제주,normal,20220204,1
3,[가지]가지,5kg,14000,경남 밀양시,normal,20220204,1
4,[감]부유단감,10kg,30000,광주 남구,normal,20220204,1
...,...,...,...,...,...,...,...
690763,[호박]늙은호박,8kg,3000,전남 해남군,normal,20220224,8
690764,[양파]양파(일반),15kg,3500,전남 무안군,normal,20220225,8
690765,[양파]양파(일반),15kg,3500,전남 무안군,normal,20220225,8
690766,[토마토]대저토마토,2.5kg,6000,경남 김해시,normal,20220226,8


### reg_date

In [14]:
df2['reg_date'] = df2['reg_date'].astype('str')

In [15]:
df2['reg_date'] = pd.to_datetime(df2['reg_date'].astype('str'),format='%Y%m%d')

In [16]:
df2

,prd,scale,price,origin,eco,reg_date,new_class
0,[대파]대파(일반),1kg,900,전남,normal,2022-02-04,1
1,[만감]레드향,5kg,12000,제주,normal,2022-02-04,1
2,[무]마대무,20kg,6500,제주,normal,2022-02-04,1
3,[가지]가지,5kg,14000,경남 밀양시,normal,2022-02-04,1
4,[감]부유단감,10kg,30000,광주 남구,normal,2022-02-04,1
...,...,...,...,...,...,...,...
690763,[호박]늙은호박,8kg,3000,전남 해남군,normal,2022-02-24,8
690764,[양파]양파(일반),15kg,3500,전남 무안군,normal,2022-02-25,8
690765,[양파]양파(일반),15kg,3500,전남 무안군,normal,2022-02-25,8
690766,[토마토]대저토마토,2.5kg,6000,경남 김해시,normal,2022-02-26,8


### price

In [17]:
# 계산을 위해 scale 값 미리 변경
# 현재 값들은 string이므로 float 형태로 변환할 필요가 있음
# 몇몇 이상치 처리도 진행
def scale_checker(scale):
  if scale.startswith('.')==True:
    return format(float('0'+scale[:-2]),'.2f')
  else:
    return format(float((scale[:-2])),'.2f')
df2['scale'] = df2['scale'].apply(scale_checker)
df2['scale'] = df2['scale'].astype('float')
# df2['scale'] = format(round(df2['scale'].apply(pd.to_numeric),2),'.2f')

In [18]:
df2['scale'].unique()

array([1.000e+00, 5.000e+00, 2.000e+01, 1.000e+01, 4.000e+00, 2.000e+00,
       1.500e+00, 3.000e+00, 7.000e-01, 3.600e+00, 5.000e-01, 4.500e+00,
       7.000e+00, 6.000e+00, 2.500e+00, 2.000e-01, 8.000e+00, 9.000e+00,
       4.000e-01, 1.700e+01, 3.000e-01, 2.500e-01, 1.200e+00, 1.400e+01,
       1.300e+01, 1.200e+01, 1.100e+01, 8.000e-01, 8.500e+00, 1.150e+01,
       8.200e+00, 7.500e+00, 9.000e-01, 1.500e+01, 7.300e+00, 2.500e+01,
       1.350e+01, 5.200e+00, 3.500e+00, 1.500e-01, 6.000e-02, 1.800e+01,
       5.000e-02, 2.700e+00, 1.700e-01, 1.200e-01, 6.000e-01, 2.010e+00,
       1.010e+00, 1.600e+01, 1.900e+01, 2.100e+01, 2.200e+01, 2.300e+01,
       8.700e+01, 1.620e+01, 1.440e+01, 1.250e+01, 5.500e+00, 3.800e+00,
       1.000e-01, 1.050e+01, 5.010e+00, 3.300e+00, 2.800e+00, 3.700e+00,
       5.600e+00, 1.080e+01, 9.600e+00, 9.800e+00, 8.010e+00, 2.400e+00,
       3.100e+00, 1.001e+01, 1.400e+00, 1.300e+00, 1.800e+00, 2.800e+01,
       5.800e+00, 8.600e+00, 2.200e+00, 4.400e+00, 

In [19]:
# 1원짜리 거래 발견, 너무 거래값이 작은 값들은 삭제
pd.options.display.float_format = '{:.5f}'.format
df2['price'].describe()

count     690768.00000
mean       23263.31064
std        80599.76642
min            0.00000
25%         8500.00000
50%        15000.00000
75%        27000.00000
max     36036000.00000
Name: price, dtype: float64

In [20]:
df2 = df2.drop(df2[df2['price']<500].index)

In [21]:
df2

,prd,scale,price,origin,eco,reg_date,new_class
0,[대파]대파(일반),1.00000,900,전남,normal,2022-02-04,1
1,[만감]레드향,5.00000,12000,제주,normal,2022-02-04,1
2,[무]마대무,20.00000,6500,제주,normal,2022-02-04,1
3,[가지]가지,5.00000,14000,경남 밀양시,normal,2022-02-04,1
4,[감]부유단감,10.00000,30000,광주 남구,normal,2022-02-04,1
...,...,...,...,...,...,...,...
690763,[호박]늙은호박,8.00000,3000,전남 해남군,normal,2022-02-24,8
690764,[양파]양파(일반),15.00000,3500,전남 무안군,normal,2022-02-25,8
690765,[양파]양파(일반),15.00000,3500,전남 무안군,normal,2022-02-25,8
690766,[토마토]대저토마토,2.50000,6000,경남 김해시,normal,2022-02-26,8


In [22]:
# kg 당 가격 생성

df2['price_kg'] = round((df2['price']/df2['scale']),2)

### prd
- 품목명 정리
- 품목 종류 정리

In [23]:
# 대부분 [상품 종류]상품명 으로 정리되어 있는걸 알 수 있음
df2['prd'].value_counts()

[딸기]설향              46789
[딸기]설향(딸기)          36798
[오이]백다다기            17058
[깻잎]깻잎              14024
[딸기]딸기(설향)          13073
                    ...  
[마]마                    1
[키조개꼭지국산]키조개꼭지국산        1
[부추]호부추(서양부추)           1
[피망]오렌지파프리카             1
[두류]강낭콩                 1
Name: prd, Length: 1158, dtype: int64

In [24]:
# 정리를 위해 상품명 삭제
def prd_sort(words):
  p = re.compile('[ㄱ-힣]+')
  word = re.search(p,words)
  return word.group()

df2['prd'] = df2['prd'].apply(prd_sort)

In [25]:
# 기타 품목 삭제
df2 = df2.drop(df2[df2['prd']=='기타'].index)

In [26]:
# 국산,수입산,깐,기타 삭제
def prd_sort2(words):
  p = re.compile('[^국산수입깐기타]+')
  word = re.search(p,str(words))
  return word.group()

df2["prd"] = df2["prd"].apply(prd_sort2)


In [27]:
# 몇몇 품목들 정리가 필요
a = sorted(df2['prd'].unique())
for i in range(len(a)):
  if i%15 != 0:
    print(a[i],end=' ')
  else:
    print(a[i],end=' ')
    print()

가리비 
가물치 가오리 가자미 가지 각굴 갈치 감 감귤 감말랭이 감자 갑오징어 갓 강낭콩 개조개 건고추 
건채류 게 게지 겨자 고 고구마 고구마순 고둥 고들빼 고등어 고사리 고추 곤달비 곤드레나물 골뱅이 
곶감 과실류 과일 과일류 과채류 굴 귤 근대 근채류 금감 김 깻잎 꼬막 꼴뚜 꽁치 
꽃게살 꽈리고추 낙지 냉이 넙치 논우렁이 눈볼대 느 다래 단감 달래 닭새우 당귀 당근 대구 
대구머리 대추 대파 더덕 도다리 도라지 돗나물 동자개 동죽 두류 두릅 두리안 딸 땅콩 람부탄 
레몬 리치 마 마늘 만가닥 만감 망고 매생이 머위대 메 메론 명태 명태곤니 명태알 명태포 
모시조개 모자반 무 무순 무우 무청 문어 문어바지락 미꾸라지 미나리 미더덕 미역 민들레 민물새우 바나나 
바지락 박 밤 방울토마토 방풍 방풍나물 배 배추 백새우 백합 뱀장어 버섯 버섯류 버찌 벅굴 
벗 보리 복숭아 봄동 봄동배추 봉지굴 부세 부지갱이 부추 붉은고추 붕어 붕장어 브로커리 브로코리 블루베리 
비 비트 빈스 빙어 사과 삼치 상추 새꼬막 새송이 새싹 새우 새우살 새조개 샐러리 생강 
생고추 석류 세러리 셀러리 소라 속새 숙주나물 시금치 실파 쌈채류 쌈추 쑥 쑥갓 씀바귀 아귀 
아로니아 아보카도 아보카드 아스파라가스 아스파라거스 아욱 알 알배 야자 야콘 양배추 양상추 양송이 양채 양채류 
양파 얼갈이 얼갈이배추 연근 연어 열무 염고등어 엽경채류 엽채 엽채류 오렌지 오만둥이 오이 오징어 오징어몸통 
오징어채 완두콩 용과 우렁쉥이 우뭇가시리 유채 은행 임연 잉어 자라 자몽 장어 재첩 적근대 적새우 
적채 전복 절단 절단낙지 준치 쪽파 쭈꾸미 참꼬막 참나물 참다래 참외 참조 채소 청경채 청어 
체리 총각무 취나물 치커리 칠게 칼라후라워 칼리플라워 칼리후라워 케일 코라비 코코넛 콜라비 콩 콩나물 키위 
키조개 키조개꼭지 키조개날개 토란 토란대 토마토 톳 파 파래김 파세리 파인애플 파프리카 팥 패류 팽이 
팽이버섯 포도 포장바지락 표고 표고버섯 풋고추 피망 피뿔고동 피조개 해삼 해파리 향나물 호두 호박 

In [28]:
### 예시
# df2[(df2['prd']=='유채') & (df2['price']==4500)]['prd'].unique()
# df2[df2['prd']=='유채']['prd'].unique()

In [29]:
### 원래 단어 뭐였는지 검색
# org_df[org_df['품목명'].str.contains('느')]['품목명'].unique()

In [30]:
### 현재 이름 검색
# df2[df2['prd']=='느']

In [33]:
###
# JG

# 고 -> 고수로 변경
df2.loc[df2["prd"]=="고","prd"] = "고수"
# 과실류, 과일류, 과채, 채소 -> 과채류로 통합
df2.loc[df2["prd"]=="과실류","prd"] = "과채류"
df2.loc[df2["prd"]=="과일류","prd"] = "과채류"
df2.loc[df2["prd"]=="과일야채류","prd"] = "과채류"
df2.loc[df2["prd"]=="과일과채류","prd"] = "과채류"
df2.loc[df2["prd"]=="과일","prd"] = "과채류"
df2.loc[df2["prd"]=="과채","prd"] = "과채류"
df2.loc[df2["prd"]=="채소","prd"] = "과채류"
# 고들빼 -> 고들빼기로 변경
df2.loc[df2["prd"]=="고들빼","prd"] = "고들빼기"
# 꼴뚜 -> 꼴두기로 변경
df2.loc[df2["prd"]=="꼴뚜","prd"] = "꼴뚜기"
# 그레이프푸룻 -> 자몽으로 통합
df2.loc[df2["prd"]=="그레이프푸룻","prd"] = "자몽"

# 느 -> 느타리버섯으로 통합
df2.loc[df2["prd"]=="느","prd"] = "느타리버섯"

# 대구 관련 품목들 정리
df2.loc[df2["prd"]=="대구고니","prd"] = "대구"
df2.loc[df2["prd"]=="대구머리","prd"] = "대구"
df2.loc[df2["prd"]=="대구포","prd"] = "대구"
df2.loc[df2["prd"]=="대구알","prd"] = "대구"
# 듀리안 -> 두리안으로 통합
df2.loc[df2["prd"]=="듀리안","prd"] = "두리안"
# 딸 -> 딸기로 변경
df2.loc[df2["prd"]=="딸","prd"] = "딸기"

# 만가닥 -> 만가닥버섯으로 변경
df2.loc[df2["prd"]=="만가닥","prd"] = "만가닥버섯"
# 맛 -> 맛조개로 변경
df2.loc[df2["prd"]=="맛","prd"] = "맛조개"
# 망고스턴 -> 망고스틴으로 수정
df2.loc[df2["prd"]=="망고스턴","prd"] = "망고스틴"
# 메 -> 메기로 변경
df2.loc[df2["prd"]=="메","prd"] = "메기"
# 명태 관련 품목들 정리
df2.loc[df2["prd"]=="명태곤니","prd"] = "명태"
df2.loc[df2["prd"]=="명태알","prd"] = "명태"
df2.loc[df2["prd"]=="명태포","prd"] = "명태"
df2.loc[df2["prd"]=="코다리명태","prd"] = "명태"
# 무우 -> 무로 통합
df2.loc[df2["prd"]=="무우","prd"] = "무"

# 방풍 -> 방풍나물로 통합
df2.loc[df2["prd"]=="방풍","prd"] = "방풍나물"
# 버섯류 -> 버섯으로 변경
df2.loc[df2["prd"]=="버섯","prd"] = "버섯"
# 벗,버찌 -> 체리로 통합
df2.loc[df2["prd"]=="벗","prd"] = "체리"
df2.loc[df2["prd"]=="버찌","prd"] = "체리"
df2.loc[df2["prd"]=="벗찌","prd"] = "체리"
# 브로커리, 브로코리 -> 브로콜리로 통합
df2.loc[df2["prd"]=="브로커리","prd"] = "브로콜리"
df2.loc[df2["prd"]=="브로코리","prd"] = "브로콜리"

# 삼 -> 수삼으로 변경
df2.loc[df2["prd"]=="삼","prd"] = "수삼"
# 새우살 -> 새우로 변경
df2.loc[df2["prd"]=="새우살","prd"] = "새우"
# 세러리, 셀러리 -> 샐러리로 통합
df2.loc[df2["prd"]=="세러리","prd"] = "샐러리"
df2.loc[df2["prd"]=="셀러리","prd"] = "샐러리"
# 생고추 -> 고추로 통합
df2.loc[df2["prd"]=="생고추","prd"] = "고추"

# 아보카드 -> 아보카도로 변경
df2.loc[df2["prd"]=="아보카드","prd"] = "아보카도"
# 아스파라가스 -> 아스파라거스로 변경
df2.loc[df2["prd"]=="아스파라가스","prd"] = "아스파라거스"
# 알 -> 알타리무로 변경
df2.loc[df2["prd"]=="알","prd"] = "알타리무"
# 알배 -> 알배기로 변경
df2.loc[df2["prd"]=="알배","prd"] = "알배기"
# 양채 -> 양채류로 통합
df2.loc[df2["prd"]=="양채","prd"] = "양채류"
# 얼갈이 -> 얼갈이배추로 통합
df2.loc[df2["prd"]=="얼갈이","prd"] = "얼갈이배추"
# 엽채류, 엽채 -> 엽경채류로 통합
df2.loc[df2["prd"]=="엽채류","prd"] = "엽경채류"
df2.loc[df2["prd"]=="엽채","prd"] = "엽경채류"
# 오징어 관련 품목들 정리
df2.loc[df2["prd"]=="오징어다리","prd"] = "오징어"
df2.loc[df2["prd"]=="오징어원양","prd"] = "오징어"
df2.loc[df2["prd"]=="오징어채","prd"] = "오징어"
df2.loc[df2["prd"]=="활복오징어","prd"] = "오징어"
df2.loc[df2["prd"]=="오징어채원양","prd"] = "오징어"
df2.loc[df2["prd"]=="갑오징어살","prd"] = "오징어"
df2.loc[df2["prd"]=="갑오징어몸통","prd"] = "오징어"
# 옥, 풋옥 -> 옥수수로 정리
df2.loc[df2["prd"]=="옥","prd"] = "옥수수"
df2.loc[df2["prd"]=="풋옥","prd"] = "옥수수"
# 임연 -> 임연수어로 정리
df2.loc[df2["prd"]=="임연","prd"] = "임연수어"


# 절단 -> 절단쭈꾸미 -> 쭈꾸미로 정리
df2.loc[df2["prd"]=="절단","prd"] = "쭈꾸미"
# 조개살 -> 조개로 정리
df2.loc[df2["prd"]=="조개살","prd"] = "조개"

# 참조 -> 참조기로 정리
df2.loc[df2["prd"]=="참조","prd"] = "참조기"

# 칼라후라워,칼리플라워, 칼리후라워 -> 컬리플라워로 통합
df2.loc[df2["prd"]=="칼라후라워","prd"] = "컬리플라워"
df2.loc[df2["prd"]=="칼리플라워","prd"] = "컬리플라워"
df2.loc[df2["prd"]=="칼리후라워","prd"] = "컬리플라워"
# 코라비 -> 콜라비로 통합
df2.loc[df2["prd"]=="코라비","prd"] = "콜라비"
# 키조개날개 -> 키조개로 통합
df2.loc[df2["prd"]=="키조개날개","prd"] = "키조개"

# 팽이 -> 팽이버섯으로 통합
df2.loc[df2["prd"]=="팽이","prd"] = "팽이버섯"
# 파세리 -> 파슬리로 변경
df2.loc[df2["prd"]=="파세리","prd"] = "파슬리"
# 표고 -> 표고버섯으로 통합
df2.loc[df2["prd"]=="표고","prd"] = "표고버섯"
# 포장바지락 -> 바지락으로 변경
df2.loc[df2["prd"]=="포장바지락","prd"] = "바지락"

### 일단 여기까지 실행
# JY
#각굴 -> 굴로 통합
df2.loc[df2["prd"]=="각굴","prd"] = "굴"
#가오리채 -> 가오리로 통합
df2.loc[df2["prd"]=="가오리채","prd"] = "가오리"
# 게지 -> 키조개로 통합
df2.loc[df2["prd"]=="게지","prd"] = "키조개"
#고 -> 고구마로 통합
df2.loc[df2["prd"]=="고","prd"] = "고구마"
#꼴뚜 -> 꼴뚜기로 통합
df2.loc[df2["prd"]=="꼴뚜","prd"] = "꼴뚜기"
#돗나물 -> 돌나물로 통합
df2.loc[df2["prd"]=="돗나물","prd"] = "돌나물"
#레드쉬 -> 레디쉬로 통합
df2.loc[df2["prd"]=="레드쉬","prd"] = "레디쉬"
#만가닥 -> 만가닥버섯으로 통합
df2.loc[df2["prd"]=="만가닥","prd"] = "만가닥버섯"
#메 -> 메기로 통합
df2.loc[df2["prd"]=="메","prd"] = "메기"
#봄동 -> 봄동배추로 통합
df2.loc[df2["prd"]=="봄동","prd"] = "봄동배추"
#봉지굴 -> 굴로 통합
df2.loc[df2["prd"]=="봉지굴","prd"] = "굴"
#박 -> 수박으로 통합
df2.loc[df2["prd"]=="박","prd"] = "수박"
#백조 -> 조기로 통합 
df2.loc[df2["prd"]=="백조","prd"] = "조기"
#벅굴 -> 벚굴로 통합
df2.loc[df2["prd"]=="벅굴","prd"] = "벚굴"
#비 -> 비타민으로 통합
df2.loc[df2["prd"]=="비","prd"] = "비타민"
#빈스 -> 콩으로 통합
df2.loc[df2["prd"]=="빈스","prd"] = "콩"
#삐틀이 -> 고둥으로 통합
df2.loc[df2["prd"]=="삐틀이","prd"] = "고둥"
#새송이 -> 새송이버섯으로 통합
df2.loc[df2["prd"]=="새송이","prd"] = "새송이버섯"
#세트 -> 과채류로 통합
df2.loc[df2["prd"]=="세트","prd"] = "과채류"
#알 -> 알로애로 통합
df2.loc[df2["prd"]=="알","prd"] = "알로에"
#엄나무 -> 음나무로 통합
df2.loc[df2["prd"]=="엄나무","prd"] = "음나무"
#염고등어 -> 고등어로 통합
df2.loc[df2["prd"]=="염고등어","prd"] = "고등어"
#오만둥이 -> 미더덕으로 통합
df2.loc[df2["prd"]=="오만둥이","prd"] = "미더덕"
#오징어 관련 품목들 정리
df2.loc[df2["prd"]=="선동오징어","prd"] = "오징어"
df2.loc[df2["prd"]=="오징어몸통","prd"] = "오징어"
#우렁쉥이 -> 멍게로 통합
df2.loc[df2["prd"]=="우렁쉥이","prd"] = "멍게"
#우뭇가시리 -> 우뭇가사리로 통합
df2.loc[df2["prd"]=="우뭇가시리","prd"] = "우뭇가사리"
#적채 -> 적양배추로 통합
df2.loc[df2["prd"]=="적채","prd"] = "적양배추"
#절단낙지 -> 낙지로 통합
df2.loc[df2["prd"]=="절단낙지","prd"] = "낙지"
#조 -> 조개로 통합
df2.loc[df2["prd"]=="조","prd"] = "조개"
#청어원양 -> 청어로 통합
df2.loc[df2["prd"]=="청어원양","prd"] = "청어"
#키조개꼭지 -> 키조개로 통합
df2.loc[df2["prd"]=="키조개꼭지","prd"] = "키조개"

# 꽃게 관련 꽃게로 통합
df2.loc[df2["prd"]=="암꽃게","prd"] = "꽃게"
df2.loc[df2["prd"]=="숫꽃게","prd"] = "꽃게"
df2.loc[df2["prd"]=="꽃게살","prd"] = "꽃게"
# 우렁 -> 우렁이로 통합
df2.loc[df2["prd"]=="우렁","prd"] = "우렁이"
# 알로애 -> 알로에로 변경
df2.loc[df2["prd"]=="알로애","prd"] = "알로에"
# 피마자잎 -> 피마자로 변경
df2.loc[df2["prd"]=="피미자잎","prd"] = "피마자"
# 식용허브 -> 허브로 변경
df2.loc[df2["prd"]=="식용허브","prd"] = "허브"
# 선인장열매 -> 선인장으로 변경
df2.loc[df2["prd"]=="선인장열매","prd"] = "선인장"
# 쌈추 -> 쌈채류로 변경
df2.loc[df2["prd"]=="쌈추","prd"] = "쌈채류"
# 덟은감 -> 감으로 변경
df2.loc[df2["prd"]=="덟은감","prd"] = "감"
# 피뿔고동 -> 고둥으로 변경
df2.loc[df2["prd"]=="피뿔고동","prd"] = "고둥"
# 꼬시래 -> 꼬시래기으로 변경
df2.loc[df2["prd"]=="꼬시래","prd"] = "꼬시래기"
# 연어살 -> 연어로 변경
df2.loc[df2["prd"]=="연어살","prd"] = "연어"
# 비단멍게 -> 멍게로 변경
df2.loc[df2["prd"]=="비단멍게","prd"] = "멍게"
# 물가자미 -> 가자미로 변경
df2.loc[df2["prd"]=="물가자미","prd"] = "가자미"
# 물메 -> 메기로 변경
df2.loc[df2["prd"]=="물메","prd"] = "메기"
# 붉은메 -> 메기로 변경
df2.loc[df2["prd"]=="붉은메","prd"] = "메기"
# 조미제품 삭제
df2.drop(df2[df2['prd']=='조미제품'].index,inplace=True)
df2.drop(df2[df2['prd']=='어류원양'].index,inplace=True)
df2.drop(df2[df2['prd']=='동물'].index,inplace=True)

In [34]:
# 기존 products 리스트와 비교
ex_prd = pd.read_csv("../products.csv")
df2[~df2['prd'].isin(ex_prd['prd'])]['prd'].unique()

array(['준치'], dtype=object)

In [35]:
# 물품 리스트 최신화
new_prd = pd.DataFrame(df2['prd'].unique(),columns=['prd'])
# products = ex_prd.append(new_prd[~new_prd['prd'].isin(ex_prd['prd'])],ignore_index=True)
products = ex_prd.append(new_prd,ignore_index=True)

/var/folders/nq/s4zx76p53d11lnr0t1dxpxh00000gn/T/ipykernel_3570/2619460125.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  products = ex_prd.append(new_prd,ignore_index=True)


In [36]:
products.drop(columns=['Unnamed: 0'],inplace=True)
products.drop_duplicates(inplace=True)
products.reset_index(inplace=True,drop=True)

In [37]:
# 다음 작업을 위해 저장
products.to_csv('../products.csv')

In [38]:
# 전체 거래량과 품목량
# 비교를 위해 분포 파악
# 거래량이 너무 적은 품목은 삭제할 필요성이 있음

prd_lst = (df2['prd'].value_counts()).to_frame()
prd_lst.describe()

,prd
count,229.00000
mean,3015.00437
std,11207.86402
min,1.00000
25%,8.00000
50%,147.00000
75%,1372.00000
max,138668.00000


In [39]:
# 50개 미만 품목은 비교가 힘드므로 삭제
# 줄어든 종류만 남기기

prd_lst = prd_lst[prd_lst['prd']>=50]
prd_lst.reset_index(inplace=True)
prd_lst = prd_lst.rename(columns = {'index':'prd','prd':'count'})

In [40]:
df3 = df2.merge(prd_lst)
df3[df3['count']>=50]['prd'].value_counts()

딸기        138668
버섯         56794
토마토        42232
오이         32896
깻잎         28072
           ...  
참나물           68
김             67
아스파라거스        67
뱀장어           57
완두콩           57
Name: prd, Length: 128, dtype: int64

In [41]:
# 줄어든 것 확인했으므로 count column은 삭제
df3 = df3.drop(columns='count')

### scale

In [42]:
np.set_printoptions(precision=2, suppress=True)
df3['scale'].unique()

array([  1.  ,  10.  ,   0.5 ,   5.  ,   3.  ,   4.5 ,   9.  ,   2.  ,
         8.  ,   6.3 ,  12.6 ,  20.  ,   4.  ,   0.8 ,  15.  ,   1.3 ,
        16.  ,   0.9 ,   6.  ,   0.6 ,   5.01,  14.  ,  13.  ,   0.3 ,
        12.  ,  16.2 ,   7.5 ,  14.4 ,  10.8 ,  28.  ,  11.7 ,   4.4 ,
        17.  ,   3.7 ,   4.3 ,   2.5 ,   2.7 ,   2.6 ,   1.5 ,   0.7 ,
         2.01,   1.01,   1.2 ,   7.  ,   0.33,   0.25,  22.  ,   0.08,
         1.4 ,   0.75,  32.  ,   3.5 ,   2.45,   3.6 ,   3.8 ,   3.3 ,
        12.5 ,  11.  ,   3.2 ,   4.7 ,   8.5 ,   5.5 ,   0.65,  10.01,
         9.6 ,  11.2 ,   4.2 ,   0.2 ,   0.15,   0.05,   0.1 ,  10.5 ,
       134.  ,   1.6 ,   0.55,   8.4 ,  25.  ,   6.5 ,   9.5 ,   0.4 ,
        18.  ,  17.23,  11.5 ,   5.4 ,   1.8 ,   8.2 ,   7.3 ,   8.6 ,
         7.8 ,  21.  ,  19.  ,  13.5 ,   5.2 , 113.  , 131.  ,  60.  ,
         4.8 ,  12.4 ,  30.  ,  45.  ,  52.  ,   9.4 ,  56.  ,  27.  ,
        11.4 ,  13.4 ,   0.16,   8.01,   0.17,   0.12,  23.  ,   0.03,
      

In [43]:
df3[df3['scale']>=50]

,prd,scale,price,origin,eco,reg_date,new_class,price_kg
254335,양채류,134.00000,14000,제주 제주시,normal,2022-02-10,1,104.48000
254336,양채류,134.00000,8500,제주 제주시,normal,2022-02-10,1,63.43000
254337,양채류,134.00000,14000,제주 제주시,normal,2022-02-10,1,104.48000
254339,양채류,134.00000,14000,제주 제주시,normal,2022-02-10,1,104.48000
328924,바나나,113.00000,28000,필리핀,normal,2022-02-15,1,247.79000
330689,바나나,131.00000,23000,콜롬비아,normal,2022-02-22,1,175.57000
331754,바나나,60.00000,26000,필리핀,normal,2022-02-26,1,433.33000
348207,피망,52.00000,296400,경남 진주시,certified,2022-02-15,2,5700.00000
348643,피망,56.00000,319200,경남 진주시,certified,2022-02-21,2,5700.00000
494061,브로콜리,87.00000,19000,제주도 제주시,normal,2022-02-04,1,218.39000


In [44]:
# 50kg 이상 거래된 건들 이상치라 판단하고 확인 진행
df3[df3['scale']>=50]['prd'].unique()

array(['양채류', '바나나', '피망', '브로콜리'], dtype=object)

In [53]:
df3[(df3['prd']=='브로콜리')&(df3['scale']>=10)]

,prd,scale,price,origin,eco,reg_date,new_class,price_kg
494061,브로콜리,87.00000,19000,제주도 제주시,normal,2022-02-04,1,218.39000
494062,브로콜리,87.00000,13500,제주도 제주시,normal,2022-02-04,1,155.17000
494834,브로콜리,15.00000,14500,제주 제주시,normal,2022-02-07,1,966.67000
495036,브로콜리,10.00000,34000,강원,certified,2022-02-08,1,3400.00000
495416,브로콜리,88.00000,12000,제주도 제주시,normal,2022-02-09,1,136.36000
495417,브로콜리,88.00000,23000,제주도 제주시,normal,2022-02-09,1,261.36000
495418,브로콜리,88.00000,13500,제주도 제주시,normal,2022-02-09,1,153.41000
495606,브로콜리,16.00000,22500,제주 제주시,normal,2022-02-10,1,1406.25000
495623,브로콜리,16.00000,21500,제주 제주시,normal,2022-02-10,1,1343.75000
495673,브로콜리,16.00000,22500,제주 제주시,normal,2022-02-10,1,1406.25000


In [54]:
# 스케일 오표기된 항목 수정
# 피망은 정상 거래
# 이상치 항목 삭제
df3 = df3.drop(df3[(df3['scale']>=50)&(df3['prd']!='피망')].index)

In [55]:
df3[df3['scale']>=50]

,prd,scale,price,origin,eco,reg_date,new_class,price_kg
348207,피망,52.00000,296400,경남 진주시,certified,2022-02-15,2,5700.00000
348643,피망,56.00000,319200,경남 진주시,certified,2022-02-21,2,5700.00000


### origin

In [56]:
# 도,시, 국산, 수입 표기 분류 필요
df3['origin'].unique()

array(['전남', '충남 서산시', '전남 신안군', '전남 진도군', '충북 청주시', '충북', '경기', '제주 제주시',
       '경기 남양주시', '경기 고양시 덕양구', '강원도 태백시', '경기 양주시', '경기 고양시덕양구',
       '인천시 계양구', '경기도 고양시', '제주', '제주 서귀포시(2)', '제주 서귀포시', '제주 남제주군',
       '제주도 서귀포시', '제주시 한림읍', '제주특별자치도 서귀', '제주 북제주군', '제주도 제주시',
       '제주시 한경면', '전남 무안군', '전남 나주시', '경기 포천시', '서울 도봉구', '경기 성남시',
       '경기 안성시', '경기 여주시', '경남 창원시', '경기 광주시', '경기 양평군', '경기 이천시',
       '경기 구리시', '경기 파주시', '경기 가평군', '전남 영암군', '경기 고양시', '전남 고흥군',
       '경기 용인시', '서울 강동구', '성남시 중원구', '충남', '충남 예산군', '강원 평창군', '충남 당진시',
       '경기 의정부시', '제주 서귀포', '제주특별자치', '경남 밀양시', '경남 창녕군', '광주시 광산구',
       '경남 김해시', '광주 광산구', '경남 고성군', '경남 함안군', '경북 상주시', '전북 군산시',
       '전북 완주군', '전북 임실군', '경기 여주군', '전북 전주시', '경남 진주시', '경남 밀양시 단장면',
       '경남 의령군', '경기 안성시 일죽면', '경남 거제시', '경남 창원시 의창구', '충남 부여군', '경북 성주군',
       '충북 충주시', '강원 춘천시', '충남 서천군', '경남 진주시 금곡면', '서울시 강서구', '충남 금산군',
       '경남', '경남 밀양시 무안면', '광주 남구', '경남 하동군', '경북 청도군', '전남 해남군',
       '경남 합천군', '전남 장성군',

In [57]:
# 해당 조건에 맞는 값 추출

splits = df3['origin'].str.split(' ')
cacul = splits.apply(lambda x: pd.Series(x))
df66 = pd.merge(df3,cacul, left_index=True, right_index=True, how='inner')
df66

,prd,scale,price,origin,eco,reg_date,new_class,price_kg,0,1,2,3
0,대파,1.00000,900,전남,normal,2022-02-04,1,900.00000,전남,NaN,NaN,NaN
1,대파,10.00000,17600,충남 서산시,normal,2022-02-04,1,1760.00000,충남,서산시,NaN,NaN
2,대파,10.00000,16800,전남 신안군,normal,2022-02-04,1,1680.00000,전남,신안군,NaN,NaN
3,대파,1.00000,950,전남 신안군,normal,2022-02-04,1,950.00000,전남,신안군,NaN,NaN
4,대파,1.00000,950,전남 신안군,normal,2022-02-04,1,950.00000,전남,신안군,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
689149,쭈꾸미,2.00000,19500,수입 중국\t,normal,2022-02-26,3,9750.00000,수입,중국\t,NaN,NaN
689150,쭈꾸미,2.00000,19500,수입 중국\t,normal,2022-02-26,3,9750.00000,수입,중국\t,NaN,NaN
689151,쭈꾸미,2.00000,19000,수입 중국\t,normal,2022-02-26,3,9500.00000,수입,중국\t,NaN,NaN
689152,쭈꾸미,2.00000,18000,수입 중국\t,normal,2022-02-26,3,9000.00000,수입,중국\t,NaN,NaN


In [58]:
# 세부 카테고리 정리 (state, city 편집 시 여기서 시작)

df4 = df66.rename(columns = {0:'state',1:'city'})

# 2,3은 필요없는 정보이므로 삭제
df4 = df4.drop(columns=[2,3])

In [59]:
df4['state'].unique()

array(['전남', '충남', '충북', '경기', '제주', '강원도', '인천시', '경기도', '제주도', '제주시',
       '제주특별자치도', '서울', '경남', '성남시', '강원', '제주특별자치', '광주시', '광주', '경북',
       '전북', '서울시', '대구', '가락동', '전라북도', '충청남도', '수입산', '전주시', '경상북도',
       '부산', '부산시', '세종', '미국', '경상남도', '세종시', '대만', '인천', '중국', '멕시코',
       '필리핀', '베트남', '부산광역시', '캄보디아', '페루', '태국', '칠레', '타이', '청주시',
       '전라남도', '울산', '인천광역시', '과테말라', '에콰도르', '코스타리카', '콜롬비아', '에쿠아도르',
       '에콰돌', '기타외국', '뉴질랜드', '포항시', '대전', '대구시', '경북/문경시', '경남/합천군',
       '아르헨티나', '호주', '스페인', '서귀포시', '이탈리아', '수입', '국내산\t'], dtype=object)

In [60]:
# 시,도 제거
def name_sort(words):
  p = re.compile('[^시도]+')
  word = re.search(p,str(words))
  return word.group()

df4["state"] = df4["state"].apply(name_sort)

# 오타도 제거

def name_sort2(words):
  p = re.compile('[^\t]+')
  word = re.search(p,words)
  return word.group()

df4["state"] = df4["state"].apply(name_sort2)

In [61]:
df4["state"].unique()

array(['전남', '충남', '충북', '경기', '제주', '강원', '인천', '제주특별자치', '서울', '경남',
       '성남', '광주', '경북', '전북', '대구', '가락동', '전라북', '충청남', '수입산', '전주',
       '경상북', '부산', '세종', '미국', '경상남', '대만', '중국', '멕', '필리핀', '베트남',
       '부산광역', '캄보디아', '페루', '태국', '칠레', '타이', '청주', '전라남', '울산', '인천광역',
       '과테말라', '에콰', '코스타리카', '콜롬비아', '에쿠아', '에콰돌', '기타외국', '뉴질랜드', '포항',
       '대전', '경북/문경', '경남/합천군', '아르헨티나', '호주', '스페인', '서귀포', '이탈리아', '수입',
       '국내산'], dtype=object)

In [62]:
# state
# 특이케이스 제거

# 국내
df4.loc[df4["state"]=="국내산","city"] = "미상"
df4.loc[df4["state"]=="국내산","state"] = "국산"
df4.loc[df4["state"]=="원양산","city"] = "미상"

## 도단위
df4.loc[df4["state"]=="충청북","state"] = "충북"
df4.loc[df4["state"]=="충청남","state"] = "충남"
df4.loc[df4["state"]=="전라북","state"] = "전북"
df4.loc[df4["state"]=="전라남","state"] = "전남"
df4.loc[df4["state"]=="경상북","state"] = "경북"
df4.loc[df4["state"]=="경상남","state"] = "경남"

## 시단위
### 광역시

df4.loc[df4["state"]=="대전","city"] = "대전"
df4.loc[df4["state"]=="대구","city"] = "대구"
df4.loc[df4["state"]=="서울특별","city"] = "서울"
df4.loc[df4["state"]=="서울특별","state"] = "서울"
df4.loc[df4["state"]=="인천광역","city"] = "인천"
df4.loc[df4["state"]=="인천광역","state"] = "인천"
df4.loc[df4["state"]=="부산","city"] = "부산"
df4.loc[df4["state"]=="부산광역","city"] = "부산"
df4.loc[df4["state"]=="부산광역","state"] = "부산"
df4.loc[df4["state"]=="가락동","city"] = "서울"
df4.loc[df4["state"]=="가락동","state"] = "서울"
df4.loc[df4["state"]=="세종","city"] = "세종"
df4.loc[df4["state"]=="세종특별자치","city"] = "세종"
df4.loc[df4["state"]=="세종특별자치","state"] = "세종"
df4.loc[df4["state"]=="대전","city"] = "대전"
df4.loc[df4["state"]=="제주","city"] = "제주"
df4.loc[df4["state"]=="제주특별자치","city"] = "제주"
df4.loc[df4["state"]=="제주특별자치","state"] = "제주"
df4.loc[df4["state"]=="제주특별자","city"] = "제주"
df4.loc[df4["state"]=="제주특별자","state"] = "제주"
df4.loc[df4["state"]=="제주/서귀포","city"] = "제주"
df4.loc[df4["state"]=="제주/서귀포","state"] = "제주"
df4.loc[df4["state"]=="서귀포","city"] = "제주"
df4.loc[df4["state"]=="서귀포","state"] = "제주"
df4.loc[df4["state"]=="세종","city"] = "세종"
df4.loc[df4["state"]=="울산","city"] = "울산"
df4.loc[df4["state"]=="울산","state"] = "울산"

### 일반시
df4.loc[df4["state"]=="성남","city"] = "성남"
df4.loc[df4["state"]=="성남","state"] = "경기"
df4.loc[df4["state"]=="전주","city"] = "전주"
df4.loc[df4["state"]=="전주","state"] = "전북"
df4.loc[df4["state"]=="청주","city"] = "청주"
df4.loc[df4["state"]=="청주","state"] = "충북"
df4.loc[df4["state"]=="수원","city"] = "수원"
df4.loc[df4["state"]=="수원","state"] = "경기"
df4.loc[df4["state"]=="부천","city"] = "부천"
df4.loc[df4["state"]=="부천","state"] = "경기"
df4.loc[df4["state"]=="경기고양","city"] = "고양"
df4.loc[df4["state"]=="경기고양","state"] = "경기"
df4.loc[df4["state"]=="충북청주","city"] = "청주"
df4.loc[df4["state"]=="충북청주","state"] = "충북"
df4.loc[df4["state"]=="전북전주","city"] = "전주"
df4.loc[df4["state"]=="전북전주","state"] = "전북"
df4.loc[df4["state"]=="경북/문경","city"] = "문경"
df4.loc[df4["state"]=="경북/문경","state"] = "경북"
df4.loc[df4["state"]=="포항","city"] = "포항"
df4.loc[df4["state"]=="포항","state"] = "경북"
df4.loc[df4["state"]=="경북/안동","city"] = "안동"
df4.loc[df4["state"]=="경북/안동","state"] = "경북"
df4.loc[df4["state"]=="전남/무안군","city"] = "전남"
df4.loc[df4["state"]=="전남/무안군","state"] = "무안"
df4.loc[df4["state"]=="경북/구미","city"] = "구미"
df4.loc[df4["state"]=="경북/구미","state"] = "경북"
df4.loc[df4["state"]=="경북/상주","city"] = "상주"
df4.loc[df4["state"]=="경북/상주","state"] = "경북"
df4.loc[df4["state"]=="경북/영천","city"] = "영천"
df4.loc[df4["state"]=="경북/영천","state"] = "경북"
df4.loc[df4["state"]=="경남/합천군","city"] = "합천"
df4.loc[df4["state"]=="경남/합천군","state"] = "경남"
df4.loc[df4["state"]=="전남/고흥군","city"] = "고흥"
df4.loc[df4["state"]=="전남/고흥군","state"] = "전남"
df4.loc[df4["state"]=="무안","state"] = "전남"
df4.loc[df4["state"]=="서해안","city"] = "미상"
df4.loc[df4["state"]=="서해안","state"] = "국산"
df4.loc[df4["state"]=="남해안","city"] = "미상"
df4.loc[df4["state"]=="남해안","state"] = "국산"


# 외국
## 아시아
df4.loc[df4["state"]=="칠레","city"] = "칠레"
df4.loc[df4["state"]=="칠레","state"] = "수입"
df4.loc[df4["state"]=="일본","city"] = "일본"
df4.loc[df4["state"]=="일본","state"] = "수입"
df4.loc[df4["state"]=="베트남","city"] = "베트남"
df4.loc[df4["state"]=="베트남","state"] = "수입"
df4.loc[df4["state"]=="우즈베크","city"] = "우즈베키스탄"
df4.loc[df4["state"]=="우즈베크","state"] = "수입"
df4.loc[df4["state"]=="이스라엘","city"] = "이스라엘"
df4.loc[df4["state"]=="이스라엘","state"] = "수입"
df4.loc[df4["state"]=="터키","city"] = "터키"
df4.loc[df4["state"]=="터키","state"] = "수입"
df4.loc[df4["state"]=="중국","city"] = "중국"
df4.loc[df4["state"]=="중국","state"] = "수입"
df4.loc[df4["state"]=="타이","city"] = "태국"
df4.loc[df4["state"]=="타이","state"] = "수입"
df4.loc[df4["state"]=="태국","city"] = "태국"
df4.loc[df4["state"]=="태국","state"] = "수입"
df4.loc[df4["state"]=="필리핀","city"] = "필리핀"
df4.loc[df4["state"]=="필리핀","state"] = "수입"
df4.loc[df4["state"]=="우즈베키스탄","city"] = "우즈베키스탄"
df4.loc[df4["state"]=="우즈베키스탄","state"] = "수입"
df4.loc[df4["state"]=="대만","city"] = "대만"
df4.loc[df4["state"]=="대만","state"] = "수입"
df4.loc[df4["state"]=="베트남","city"] = "베트남"
df4.loc[df4["state"]=="베트남","state"] = "수입"
df4.loc[df4["state"]=="이란","city"] = "이란"
df4.loc[df4["state"]=="이란","state"] = "수입"
df4.loc[df4["state"]=="러","city"] = "러시아"
df4.loc[df4["state"]=="러","state"] = "수입"
df4.loc[df4["state"]=="캄보디아","city"] = "캄보디아"
df4.loc[df4["state"]=="캄보디아","state"] = "수입"
df4.loc[(df4["state"]=="인")&(df4["origin"]=="인도네시아"),"city"] = "인도네시아"
df4.loc[(df4["state"]=="인")&(df4["origin"]=="인도네시아"),"state"] = "수입"
df4.loc[(df4["state"]=="인")&(df4["origin"]=="인도"),"city"] = "인도"
df4.loc[(df4["state"]=="인")&(df4["origin"]=="인도"),"state"] = "수입"

## 아메리카

df4.loc[df4["state"]=="캘리포니아","city"] = "미국"
df4.loc[df4["state"]=="캘리포니아","state"] = "수입"
df4.loc[df4["state"]=="포클랜드","city"] = "영국"
df4.loc[df4["state"]=="포클랜드","state"] = "수입"
df4.loc[df4["state"]=="콜롬비아","city"] = "콜롬비아"
df4.loc[df4["state"]=="콜롬비아","state"] = "수입"
df4.loc[df4["state"]=="에콰","city"] = "에콰도르"
df4.loc[df4["state"]=="에콰","state"] = "수입"
df4.loc[df4["state"]=="에쿠아","city"] = "에콰도르"
df4.loc[df4["state"]=="에쿠아","state"] = "수입"
df4.loc[df4["state"]=="에콰돌","city"] = "에콰도르"
df4.loc[df4["state"]=="에콰돌","state"] = "수입"
df4.loc[df4["state"]=="과테말라","city"] = "과테말라"
df4.loc[df4["state"]=="과테말라","state"] = "수입"
df4.loc[df4["state"]=="코스타리카","city"] = "코스타리카"
df4.loc[df4["state"]=="코스타리카","state"] = "수입"
df4.loc[df4["state"]=="멕","city"] = "멕시코"
df4.loc[df4["state"]=="멕","state"] = "수입"
df4.loc[df4["state"]=="페루","city"] = "페루"
df4.loc[df4["state"]=="페루","state"] = "수입"
df4.loc[df4["state"]=="미국","city"] = "미국"
df4.loc[df4["state"]=="미국","state"] = "수입"
df4.loc[df4["state"]=="아르헨티나","city"] = "아르헨티나"
df4.loc[df4["state"]=="아르헨티나","state"] = "수입"
df4.loc[df4["state"]=="브라질","city"] = "브라질"
df4.loc[df4["state"]=="브라질","state"] = "수입"

## 유럽
df4.loc[df4["state"]=="스페인","city"] = "스페인"
df4.loc[df4["state"]=="스페인","state"] = "수입"
df4.loc[df4["state"]=="우크라이나","city"] = "우크라이나"
df4.loc[df4["state"]=="우크라이나","state"] = "수입"
df4.loc[df4["state"]=="이탈리아","city"] = "이탈리아"
df4.loc[df4["state"]=="이탈리아","state"] = "수입"
df4.loc[df4["state"]=="불가리아","city"] = "불가리아"
df4.loc[df4["state"]=="불가리아","state"] = "수입"

## 아프리카
df4.loc[df4["state"]=="남아프리카","city"] = "남아공"
df4.loc[df4["state"]=="남아프리카","state"] = "수입"
df4.loc[df4["state"]=="남아프리카공화국","city"] = "남아공"
df4.loc[df4["state"]=="남아프리카공화국","state"] = "수입"
df4.loc[df4["state"]=="수입남아프리카공화국","city"] = "남아공"
df4.loc[df4["state"]=="수입남아프리카공화국","state"] = "수입"


## 오세아니아
df4.loc[df4["state"]=="뉴질랜드","city"] = "뉴질랜드"
df4.loc[df4["state"]=="뉴질랜드","state"] = "수입"
df4.loc[df4["state"]=="오스트레일","city"] = "호주"
df4.loc[df4["state"]=="오스트레일","state"] = "수입"
df4.loc[df4["state"]=="오스트레일리아","city"] = "호주"
df4.loc[df4["state"]=="오스트레일리아","state"] = "수입"
df4.loc[df4["state"]=="호주","city"] = "호주"
df4.loc[df4["state"]=="호주","state"] = "수입"
df4.loc[df4["state"]=="통가","city"] = "통가"
df4.loc[df4["state"]=="통가","state"] = "수입"
df4.loc[df4["state"]=="뉴칼레","city"] = "뉴칼레도니아"
df4.loc[df4["state"]=="뉴칼레","state"] = "수입"
df4.loc[df4["state"]=="뉴","city"] = "뉴칼레도니아"
df4.loc[df4["state"]=="뉴","state"] = "수입"
df4.loc[df4["state"]=="누벨칼레","city"] = "뉴칼레도니아"
df4.loc[df4["state"]=="누벨칼레","state"] = "수입"
# 기타
df4.loc[df4["state"]=="수입산","state"] = "수입"
df4.loc[df4["state"]=="기타외국","city"] = "미상"
df4.loc[df4["state"]=="기타외국","state"] = "수입"
df4.drop(df4[df4["state"]=="미등록"].index,inplace=True)

In [63]:
df4['state'].unique()

array(['전남', '충남', '충북', '경기', '제주', '강원', '인천', '서울', '경남', '광주', '경북',
       '전북', '대구', '수입', '부산', '세종', '울산', '대전', '국산'], dtype=object)

In [64]:
# city
# 우선 결측치 제거
df4.loc[df4["city"].isnull(),"city"] = "미상"

In [65]:
# 이후 시,군 등 제거
def name_sort(words):
  p = re.compile('[^시군]+')
  word = re.search(p,words)
  return word.group()

df4["city"] = df4["city"].apply(name_sort)

In [66]:
# \t 도 제거
def name_sort2(words):
  p = re.compile('[^\t]+')
  word = re.search(p,words)
  return word.group()

df4["city"] = df4["city"].apply(name_sort2)

In [67]:
df4['city'].unique()

array(['미상', '서산', '신안', '진도', '청주', '제주', '남양주', '고양', '태백', '양주', '계양구',
       '무안', '나주', '포천', '도봉구', '성남', '안성', '여주', '창원', '광주', '양평', '이천',
       '구리', '파주', '가평', '영암', '고흥', '용인', '강동구', '예산', '평창', '당진', '의정부',
       '밀양', '창녕', '광산구', '김해', '고성', '함안', '상주', '산', '완주', '임실', '전주',
       '진주', '의령', '거제', '부여', '성주', '충주', '춘천', '서천', '강서구', '금산', '남구',
       '하동', '청도', '해남', '합천', '장성', '김천', '여수', '산청', '대구', '고창', '사천',
       '담양', '강릉', '문경', '인제', '보성', '영주', '정선', '구미', '서울', '홍천', '제천',
       '장흥', '태안', '남원', '부안', '곡성', '횡성', '고령', '양구', '중구', '김제', '논산',
       '칠곡', '영덕', '하남', '음성', '광명', '양천구', '익산', '진천', '송파구', '함양', '거창',
       '북구', '공주', '홍성', '순창', '보령', '청양', '이집트', '남해', '안산', '광양', '화성',
       '베트남', '계룡', '진안', '미국', '영동', '화천', '평택', '철원', '부산', '화순', '정읍',
       '경주', '순천', '장수', '강진', '속초', '서구', '세종', '무주', '아산', '구례', '함평',
       '괴산', '김포', '완도', '동대문구', '의왕', '경산', '영광', '영양', '중랑구', '대만',
       '포항', '흥', '강화', '천안', '원주', '논', '중국

In [75]:
# 나머지 케이스 정리

# 국내
df4.loc[df4["city"]=="기타","city"] = "미상"
## 일반시
df4.loc[df4["city"]=="위","city"] = "군위"
df4.loc[df4["city"]=="고흥(나로도)","city"] = "고흥"
df4.loc[df4["city"]=="산","city"] = "군산"
df4.loc[df4["city"]=="장항(서천)","city"] = "서천"
df4.loc[df4["city"]=="삼천포","city"] = "사천"
df4.loc[df4["city"]=="삼천포","city"] = "사천"
df4.loc[df4["city"]=="순천(광양)","city"] = "순천"
df4.loc[df4["city"]=="대천(보령)","city"] = "보령"
df4.loc[df4["city"]=="대천(보령)","city"] = "보령"
df4.loc[df4["city"]=="화성(평택)","city"] = "화성"
df4.loc[df4["city"]=="흥","city"] = "시흥"
df4.loc[df4["city"]=="목포(무안)","city"] = "목포"
df4.loc[df4["city"]=="목포(무안)","state"] = "전남"
df4.loc[df4["city"]=="보성(벌교)","city"] = "보성"
df4.loc[df4["city"]=="울진(후포)","city"] = "울진"
df4.loc[df4["city"]=="논","city"] = "논산"
df4.loc[df4["city"]=="논","city"] = "논산"
df4.loc[df4["city"]=="순","city"] = "순창"
df4.loc[df4["city"]=="익","city"] = "익산"
df4.loc[df4["city"]=="미금","city"] = "성남"
df4.loc[df4["city"]=="주문진","city"] = "강릉"
df4.loc[df4["city"]=="거제(장승포)","city"] = "거제"
df4.loc[(df4["city"]=="완")&(df4["state"]=="전북"),"city"] = "완주"
df4.loc[(df4["state"]=="전북")&(df4["city"]=="남"),"city"] = "남원"
df4.loc[(df4['prd']=="호박") & (df4['state']=='전북'),"city"] = "전주"
df4.loc[(df4["state"]=="전남")&(df4["city"]=="신"),"city"] = "신안"
df4.loc[(df4["state"]=="전북")&(df4["city"]=="장"),"city"] = "장수"
df4.loc[(df4["state"]=="전남")&(df4["city"]=="진"),"city"] = "진도"
df4.loc[(df4["state"]=="충남")&(df4["city"]=="부"),"city"] = "부여"
df4.loc[(df4["state"]=="경남")&(df4["city"]=="의"),"city"] = "의령"
df4.loc[(df4["state"]=="전남")&(df4["city"]=="함"),"city"] = "함평"
## 광역시
df4.loc[df4["city"]=="강서구","city"] = "서울"
df4.loc[df4["city"]=="은평구","city"] = "서울"
df4.loc[df4["city"]=="강동구","city"] = "서울"
df4.loc[df4["city"]=="마포구","city"] = "서울"
df4.loc[df4["city"]=="중구","city"] = "서울"
df4.loc[df4["city"]=="송파구","city"] = "서울"
df4.loc[df4["city"]=="도봉구","city"] = "서울"
df4.loc[df4["city"]=="중랑구","city"] = "서울"
df4.loc[df4["city"]=="양천구","city"] = "서울"
df4.loc[df4["city"]=="성동구","city"] = "서울"
df4.loc[df4["city"]=="강남구","city"] = "서울"
df4.loc[df4["city"]=="동대문구","city"] = "서울"
df4.loc[df4["city"]=="영등포구","city"] = "서울"
df4.loc[df4["city"]=="영등포","city"] = "서울"
df4.loc[df4["city"]=="서초구","city"] = "서울"
df4.loc[df4["city"]=="구로구","city"] = "서울"
df4.loc[df4["city"]=="동대문","city"] = "서울"
df4.loc[df4["city"]=="금천구","city"] = "서울"
df4.loc[df4["city"]=="동작구","city"] = "서울"
df4.loc[df4["city"]=="용산구","city"] = "서울"
df4.loc[df4["city"]=="광진구","city"] = "서울"
df4.loc[df4["city"]=="종로구","city"] = "서울"

df4.loc[df4["city"]=="광산구","city"] = "광주"
df4.loc[(df4["state"]=="광주")&(df4["city"]=="남구"),"city"] = "광주"
df4.loc[(df4["state"]=="광주")&(df4["city"]=="북구"),"city"] = "광주"
df4.loc[(df4["state"]=="광주")&(df4["city"]=="서구"),"city"] = "광주"
df4.loc[df4["city"]=="동구","city"] = "광주"
df4.loc[(df4["state"]=="인천")&(df4["city"]=="남구"),"city"] = "인천"
df4.loc[(df4["state"]=="인천")&(df4["city"]=="서구"),"city"] = "인천"
df4.loc[df4["city"]=="미추홀","city"] = "인천"
df4.loc[df4["city"]=="부평구","city"] = "인천"
df4.loc[df4["city"]=="남동구","city"] = "인천"
df4.loc[df4["city"]=="미추홀구","city"] = "인천"
df4.loc[df4["city"]=="계양구","city"] = "인천"
df4.loc[df4["city"]=="마산(고성,진해)","city"] = "창원"
df4.loc[(df4["state"]=="부산")&(df4["city"]=="북구"),"city"] = "부산"
df4.loc[df4["city"]=="오포","city"] = "부산"
df4.loc[df4["city"]=="부산진","city"] = "부산"
df4.loc[df4["city"]=="사상구","city"] = "부산"
df4.loc[df4["city"]=="부산진구","city"] = "부산"
df4.loc[df4["city"]=="북제주","city"] = "제주"
df4.loc[df4["city"]=="남제주","city"] = "제주"

# 해외
## 아시아
df4.loc[df4["city"]=="러","city"] = "러시아"
df4.loc[df4["city"]=="말레이지아","city"] = "말레이시아"
df4.loc[df4["city"]=="타이","city"] = "태국"
df4.loc[df4["city"]=="인도네","city"] = "인도네시아"
df4.loc[df4["city"]=="사우디","city"] = "사우디아라비아"
## 아메리카
df4.loc[df4["city"]=="멕","city"] = "멕시코"
## 아프리카
df4.loc[df4["city"]=="남아공산","city"] = "남아공"
## 유럽
## 오세아니아

In [76]:
df4[df4['city']=='함']

,prd,scale,price,origin,eco,reg_date,new_class,price_kg,state,city


In [77]:
# 기존 city 리스트와 비교
ex_city = pd.read_csv("../cities.csv")
df4[~df4['city'].isin(ex_city['city'])]['city'].unique()

array(['그리스'], dtype=object)

In [78]:
# 리스트 최신화
new_city = pd.DataFrame(df4['city'].unique(),columns=['city'])
cities = ex_city.append(new_city,ignore_index=True)

/var/folders/nq/s4zx76p53d11lnr0t1dxpxh00000gn/T/ipykernel_3570/1856950528.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cities = ex_city.append(new_city,ignore_index=True)


In [79]:
cities.drop(columns=['Unnamed: 0'],inplace=True)
cities.drop_duplicates(inplace=True)
cities.reset_index(inplace=True,drop=True)

In [80]:
cities.to_csv('../cities.csv')

In [81]:
# 처리 완료하였으니 df4는 백업으로 두고 df5 활용
df5 = df4.drop(columns='origin')

# 전처리 결과

# 파일 저장

In [82]:
df5.to_csv('../data_proceed/202202_proceed.csv')